In [1]:
from mysql_connection import get_cursor
import time
import pandas as pd
import numpy as np
from FoodItem import FoodItem


cursor = get_cursor()
# Start the timer
start = time.perf_counter()

In [2]:
user_id = (1,)
foodchecks = cursor.callproc("GetFoodChecksByID", user_id)
checks = []
# stored_results is iterable, have to do this way
for result in cursor.stored_results():
    checks = result.fetchall()[0][1:]

print(checks)

(0, 0, 0, 0, 1, 1, 1, 1, 1)


In [13]:
incoming_data = {"latitude": "50.397120", "longitude": "-4.036932", "max-distance": "10", "isForChild": 0}
foodchecks = cursor.callproc("GetRestaurantsWithinDistance", (incoming_data["latitude"], incoming_data["longitude"], incoming_data["max-distance"]))
df = None

for result in cursor.stored_results():
    df = pd.DataFrame(result)

if(df.size < 1):
    raise Exception("No restaurants found within specified distance")
df.columns = ["RestaurantID", "IsVegetarian", "IsVegan", "IsHalal", "IsKosher", "HasLactose", "HasNuts", "HasGluten", "HasEgg", "HasSoy", "FoodID", "FoodName", "FoodNameShort", "IsChildMenu", "FoodTagID"]
print(df)

    RestaurantID  IsVegetarian  IsVegan  IsHalal  IsKosher  HasLactose  \
0              2             0        0        0         0           1   
1              2             0        0        0         0           1   
2              2             0        0        0         0           1   
3              2             0        0        0         0           1   
4              2             0        0        0         0           1   
5              2             0        0        0         0           1   
6              2             0        0        0         0           1   
7              2             0        0        0         0           1   
8              2             0        0        0         0           1   
9              2             0        0        0         0           1   
10             2             0        0        0         0           1   
11             2             0        0        0         0           1   
12             2             0        

In [4]:
if checks[0]:
    df = df.drop(df[df["IsVegetarian"] != checks[0]].index)
if checks[1]:
    df = df.drop(df[df["IsVegan"] != checks[1]].index)
if checks[2]:
    df = df.drop(df[df["IsHalal"] != checks[2]].index)
if checks[3]:
    df = df.drop(df[df["IsKosher"] != checks[3]].index)
if checks[4]:
    df = df.drop(df[df["HasLactose"] != checks[4]].index)
if checks[5]:
    df = df.drop(df[df["HasNuts"] != checks[5]].index)
if checks[6]:
    df = df.drop(df[df["HasGluten"] != checks[6]].index)
if checks[7]:
    df = df.drop(df[df["HasEgg"] != checks[7]].index)
if checks[8]:
    df = df.drop(df[df["HasSoy"] != checks[8]].index)
if incoming_data["isForChild"]:
    df = df.drop(df[df["IsChildMenu"] != 1].index)

In [5]:
df

,RestaurantID,IsVegetarian,IsVegan,IsHalal,IsKosher,HasLactose,HasNuts,HasGluten,HasEgg,HasSoy,FoodID,FoodName,FoodNameShort,IsChildMenu,FoodTagID
0,2,0,0,0,0,1,1,1,1,1,2,Chicken Katsu Curry,Katsu Curry,0,5
1,2,0,0,0,0,1,1,1,1,1,2,Chicken Katsu Curry,Katsu Curry,0,6
2,2,0,0,0,0,1,1,1,1,1,2,Chicken Katsu Curry,Katsu Curry,0,7
3,2,0,0,0,0,1,1,1,1,1,2,Chicken Katsu Curry,Katsu Curry,0,8
4,2,0,0,0,0,1,1,1,1,1,5,Ackbar's Handcaught Salmon Nigiri,Salmon Nigiri,0,9
5,2,0,0,0,0,1,1,1,1,1,5,Ackbar's Handcaught Salmon Nigiri,Salmon Nigiri,0,11
6,2,0,0,0,0,1,1,1,1,1,5,Ackbar's Handcaught Salmon Nigiri,Salmon Nigiri,0,12
7,2,0,0,0,0,1,1,1,1,1,6,Vegetable Gyoza,Vegetable Gyoza,0,13
8,2,0,0,0,0,1,1,1,1,1,6,Vegetable Gyoza,Vegetable Gyoza,0,15
9,2,0,0,0,0,1,1,1,1,1,6,Vegetable Gyoza,Vegetable Gyoza,0,16


In [6]:
food_items = []

for food_id in df.FoodID.unique():
    # For each unique food id
    # Extract that part of the dataframe, and create a new food item from it
    food_items.append(FoodItem(df[df["FoodID"] == food_id]))

In [7]:
end = time.perf_counter()
print(f"Done in {end - start:0.4f} seconds")

Done in 0.3734 seconds


In [8]:
for food_item in food_items:
    food_item.printInfo()

ID: 2 - (<class 'int'>)
FoodTags: [5, 6, 7, 8] - (<class 'list'>)
Restaurant ID: 2
ID: 5 - (<class 'int'>)
FoodTags: [9, 11, 12] - (<class 'list'>)
Restaurant ID: 2
ID: 6 - (<class 'int'>)
FoodTags: [13, 15, 16, 17] - (<class 'list'>)
Restaurant ID: 2
ID: 7 - (<class 'int'>)
FoodTags: [18, 20, 21] - (<class 'list'>)
Restaurant ID: 2
ID: 8 - (<class 'int'>)
FoodTags: [14, 22, 23, 24] - (<class 'list'>)
Restaurant ID: 2
ID: 10 - (<class 'int'>)
FoodTags: [5, 7, 8, 25, 26] - (<class 'list'>)
Restaurant ID: 2
ID: 22 - (<class 'int'>)
FoodTags: [30, 31, 35] - (<class 'list'>)
Restaurant ID: 5
ID: 23 - (<class 'int'>)
FoodTags: [2] - (<class 'list'>)
Restaurant ID: 5
ID: 24 - (<class 'int'>)
FoodTags: [32] - (<class 'list'>)
Restaurant ID: 5
ID: 25 - (<class 'int'>)
FoodTags: [33, 34] - (<class 'list'>)
Restaurant ID: 5
ID: 26 - (<class 'int'>)
FoodTags: [4, 18, 32, 36] - (<class 'list'>)
Restaurant ID: 5
ID: 13 - (<class 'int'>)
FoodTags: [27, 28] - (<class 'list'>)
Restaurant ID: 5
ID: 16 